In [0]:
%%bash

apt-get install -y xvfb x11-utils python-opengl swig cmake ffmpeg freeglut3-dev

#pip install ray ray[rllib] ray[debug]
pip install Box2D box2d-py box2d-kengz gym[box2d] gym[Box_2D]\
            pyvirtualdisplay\
            PyOpenGL\
            piglet\
            piglet-templates\
            PyOpenGL-accelerate\
            stable-baselines3[extra]

In [0]:
%matplotlib inline

In [0]:
import gym
import os
import matplotlib.pyplot as plt
import numpy as np
import torch

from stable_baselines3 import PPO, SAC, TD3, A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor

# Create Continous Gym Environment




In [0]:
# LunarLanderContinuous-v2
# MountainCarContinuous-v0
# Pendulum-v0

env_name = "LunarLanderContinuous-v2"
env = gym.make(env_name)

log_dir = f"./gym/{env_name}"
os.makedirs(log_dir, exist_ok=True)
env = Monitor(env, log_dir)
callback = EvalCallback(env, log_path=log_dir, eval_freq=1000, deterministic=False )

In [0]:
total_timesteps = 1000 #300000

# SAC

In [0]:
%%time

sac_hyperparams = {
    'learning_rate': 3.0e-4,
    'buffer_size': 1000000,
    'gamma': 0.99,
    'batch_size':256,
    'tau': 0.005,
    'device':'cuda',
}


modelSAC = SAC('MlpPolicy', env, **sac_hyperparams)
modelSAC.learn(total_timesteps=total_timesteps, callback=callback)

# Save the agent

modelSAC.save(f"modelSAC_{env_name}")
del modelSAC

# Evaluate the trained agent
modelSAC = SAC.load(f"modelSAC_{env_name}")

eval_env = gym.make(env_name)

mean_reward, std_reward = evaluate_policy(modelSAC, eval_env, n_eval_episodes=100, deterministic=False)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

# PPO

In [0]:
%%time
modelPPO = PPO('MlpPolicy', env)
modelPPO.learn(total_timesteps=total_timesteps, callback=callback)

modelPPO.save(f"modelPPO_{env_name}")
del modelPPO

# Evaluate the trained agent
modelPPO = PPO.load(f"modelPPO_{env_name}")

eval_env = gym.make(env_name)

mean_reward, std_reward = evaluate_policy(modelPPO, eval_env, n_eval_episodes=100, deterministic=False)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

# TD3

In [0]:
%%time
modelTD3 = TD3('MlpPolicy', env)
modelTD3.learn(total_timesteps=total_timesteps, callback=callback)

modelTD3.save(f"modelTD3_{env_name}")
del modelTD3


# Evaluate the trained agent
modelTD3 = TD3.load(f"modelTD3_{env_name}")

eval_env = gym.make(env_name)

mean_reward, std_reward = evaluate_policy(modelTD3, eval_env, n_eval_episodes=100, deterministic=False)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

# Record agents in action


In [0]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [0]:
record_video(env_name, modelTD3, video_length=1000, prefix=f'td3_{env_name}')
record_video(env_name, modelPPO, video_length=1000, prefix=f'ppo_{env_name}')
record_video(env_name, modelSAC, video_length=1000, prefix=f'sac_{env_name}')

In [0]:
show_videos('videos', prefix=f'sac_{env_name}')

In [0]:
show_videos('videos', prefix=f'td3_{env_name}')

In [0]:
show_videos('videos', prefix=f'ppo_{env_name}')

In [0]:
from google.colab import files

files.download(f'/content/td3_{env_name}.zip')
files.download(f'/content/ppo_{env_name}.zip')
files.download(f'/content/sac_{env_name}.zip')